In [2]:
import numpy as np
import pandas as pd
import os
from function import Slope, minSSE_recovery

s_c = '結算價'
top_path = './../../../'
Data_path = top_path + 'InterpData/'
expiry = 'NearbyMonth'

IV_type = 'putIV'
K_Range = [500, 300]
K_Range_file = '{}_{}.csv'.format(K_Range[0], K_Range[1])
K_Range_dir = '{}_{}'.format(K_Range[0], K_Range[1])
IV_path = '{}/{}/{}/{}'.format(Data_path, expiry, IV_type, K_Range_file)

Dir_tree = [top_path, 'ForecastData', expiry, IV_type, K_Range_dir]
current_path = Dir_tree[0]
for i in range(1, len(Dir_tree), 1):
    if Dir_tree[i] not in os.listdir(current_path):
        os.mkdir(current_path + Dir_tree[i])
    current_path = current_path + Dir_tree[i] + '/'

IV_data = pd.read_csv(IV_path, encoding='Big5', index_col=False)
IV_matrix = np.array(IV_data)

#----------------------------------------------------------------------------


OC = np.array(IV_data['期貨開盤價'] - IV_data['期貨收盤價'])
HL = np.array(IV_data['期貨最高價'] - IV_data['期貨最低價'])
expirty_days = np.array(IV_data['到期天數'])
#OC_HL_K_E = np.vstack((OC, HL, Kmin_reduce_F, Kmax_reduce_F, expirty_days)).T
OCHL = np.vstack((OC, HL)).T


IV_matrix = np.array(IV_data)
K_num = len(np.where(IV_matrix[0, 0] == IV_matrix)[0])
K = np.array(IV_data['履約價'])
K = np.reshape(K, (-1, K_num))
IV= np.array(IV_data['隱含波動率({})'.format(s_c)])
IV = np.reshape(IV, (-1, K_num))
K_IVslope = Slope(X=K, Y=IV, axis=1)
E = IV_matrix[range(0, len(IV_matrix), K_num), IV_data.columns.get_loc('到期天數')]

In [6]:
one_day_expiry_idx = np.arange(len(E))[np.equal(E, 1)]
if one_day_expiry_idx[-1] == len(E)-1:
    one_day_expiry_idx = one_day_expiry_idx[:-1]
most_days_expiry_idx = one_day_expiry_idx + 1
most_days_expiry_idx = np.hstack((0, most_days_expiry_idx))

most_days_expiry_idx0 = most_days_expiry_idx[:-1]
most_days_expiry_idx1 = most_days_expiry_idx[1:]
trade_days_in_month = most_days_expiry_idx1 - most_days_expiry_idx0  
most_days_expiry = E[most_days_expiry_idx]

contract_appear_days1 = [list(range(1, trade_day_in_month+1, 1)) \
                for trade_day_in_month in trade_days_in_month]
contract_appear_days1 = [contract_appear_day1 for subcontract_appear_day1 in contract_appear_days1\
                         for contract_appear_day1 in subcontract_appear_day1]
contract_appear_days1 = np.array(contract_appear_days1)
contract_appear_days2 = np.arange(1, len(E) - most_days_expiry_idx[-1] +1, 1)
contract_appear_days = np.hstack((contract_appear_days1, contract_appear_days2))
#contract_appear_days 為該契約(特定交易日期、到期日期，履約價不限)上市的交易日數，例如第一天上市到期天數35天，
#則值為1，第二天則到期日為34天值為2，一直到k(因為是交易日數，所以數字不一定)。然後又到下個月的契約，值又從1開
#始
from function import TimeSeriesData
seq_length1 = 15
seq_length2 = seq_length1 + 1
min_contract_day = 2
magnification_slope = 100



#變數有考慮到期日
#K_IVslope_E = np.hstack((K_IVslope*magnification_slope, np.reshape(E, (-1, 1))))

#Inputs_slope, Ouputs = TimeSeriesData(K_IVslope_E, seq_length=seq_length1, \
                                      #drop_out_columns=[len(K_IVslope_E[0])-1])

#變數沒考慮到期日
K_IVslope_E = K_IVslope * magnification_slope
Inputs_slope, Ouputs = TimeSeriesData(K_IVslope_E, seq_length=seq_length1, drop_out_columns=[])



Inputs_OCHLE  = OCHL[range(0, len(IV_matrix), K_num)]
Inputs_OCHLE  = np.array([Inputs_OCHLE[i:i+seq_length2] \
                              for i in range(len(Inputs_OCHLE)-seq_length2+1)])
Inputs_OCHLE  = Inputs_OCHLE.astype(float)

Inputs_slope = np.reshape(Inputs_slope, (Inputs_slope.shape[0], Inputs_slope.shape[1] * Inputs_slope.shape[2]))
Inputs_OCHLE = np.reshape(Inputs_OCHLE, (Inputs_OCHLE.shape[0], Inputs_OCHLE.shape[1] * Inputs_OCHLE.shape[2]))


Inputs_OCHLE  = Inputs_OCHLE.astype(float)
if seq_length1  > seq_length2-1:
    Inputs_OCHLE = Inputs_OCHLE[seq_length1-seq_length2+1:]
    contract_appear_days = contract_appear_days[seq_length1-seq_length2+1:]
    IV_matrix_forecast = IV_matrix[seq_length1*K_num:]
if seq_length1 <= seq_length2-1:
    Inputs_slope = Inputs_slope[seq_length2-1-seq_length1:]
    Ouputs = Ouputs[seq_length2-1-seq_length1:]
    contract_appear_days = contract_appear_days[seq_length2-1:]
    IV_matrix_forecast = IV_matrix[(seq_length2-1)*K_num:]
Inputs = np.hstack((Inputs_slope, Inputs_OCHLE))

match_cond = np.greater_equal(contract_appear_days, min_contract_day)
Inputs = Inputs[match_cond]
contract_appear_days_filter = contract_appear_days[match_cond]
contract_appear_days_filter_Knum = contract_appear_days_filter.repeat(K_num)
match_cond_Knum = match_cond.repeat(K_num)
IV_matrix_forecast = IV_matrix_forecast[match_cond_Knum]
IV_matrix_forecast= np.hstack((IV_matrix_forecast, np.reshape(contract_appear_days_filter_Knum, (-1, 1))))
Ouputs = Ouputs[match_cond]



train_size = int(len(Inputs)*0.8)
#val_size = int(train_size*0.2)
X_train = Inputs[:train_size]
y_train = Ouputs[:train_size]
#y_train = Ouputs[:train_size-val_size]
#y_val = Ouputs[train_size - val_size:train_size]


X_test = Inputs[train_size:]
y_test = Ouputs[train_size:]
IV_matrix_test = IV_matrix_forecast[K_num*train_size:]


In [7]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, concatenate
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

from skopt import gp_minimize
from skopt.space import Integer, Real
from skopt.utils import use_named_args
from skopt.acquisition import gaussian_ei

#進行調參的次數
n_calls=20
#此函式可以幫我們設定一個模型的結構
def set_Linear_series(model_structure):
    neurons_num = model_structure['neurons_num']
    activations = model_structure['dense_activations']
    kernel_regularizer= model_structure['kernel_regularizer']

    input = Input(shape=model_structure['input_shape'])
    hidden = input

    for i in range(0, len(neurons_num)-1, 1):
        hidden = Dense(neurons_num[i], kernel_regularizer=kernel_regularizer[i], \
                       activation=activations[i])(hidden)
    
    output = Dense(neurons_num[-1], kernel_regularizer=kernel_regularizer[-1])(hidden)
    

    model = Model(inputs= input, outputs=output)
    return model

search_space = [
    Integer(100, 600, name='neurons_num'),
    Integer(2, 5, name='layers_num'),
    Real(1e-6, 1e-2, prior='log-uniform', name='lambda'),
]


def train_and_evaluate_net(neurons_num_, layers_num, lambda_, outputshape):
    batch_size = 500
    epochs = 1000

    neurons_num = [neurons_num_]* layers_num
    neurons_num[-1] = outputshape 
    kernel_regularizer=[l2(lambda_)]*len(neurons_num)

    model_structure = {
        'input_shape': X_train.shape[1],
        'neurons_num': neurons_num,
        'kernel_regularizer':kernel_regularizer,
        'dense_activations': ['tanh'] * layers_num
        }

    alltrain = 1
    allMSE = np.zeros(alltrain) 
    for j in range(alltrain):
        #設定模型結構
        model = set_Linear_series(model_structure)
        model.compile(loss='mse', optimizer = 'adam', metrics=['mse'])  

        # 訓練模型
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=10, \
                                verbose=2, mode='min', restore_best_weights=True)
        hist_model = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, \
            validation_split=0.2, callbacks=[early_stopping]
        )

        #得到輸出
        y_pred = model.predict(X_test)
        y_pred = y_pred / magnification_slope
        columns_names = ['履約價', '隱含波動率({})'.format(s_c)]
        column_index = [IV_data.columns.get_loc(col) for col in columns_names]

        SSE_everyday = np.zeros(((len(y_pred), K_num)))
    
        for i in range(len(y_pred)):
            K_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[0]]
            iv_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[1]]
            y_pred_day = y_pred[i]

            ForecastData, SSE_everyday[i] = minSSE_recovery(y=iv_day, x=K_day, slope_yhat=y_pred_day)
        sse_everyday = SSE_everyday[:, 0]
        allMSE[j] = np.mean(sse_everyday)

    loss = np.min(allMSE)
    return loss

@use_named_args(search_space)
def objective(**params):
    neurons_num_ = params['neurons_num']
    layers_num = params['layers_num']
    lambda_ = params['lambda']
    loss = train_and_evaluate_net(neurons_num_=neurons_num_, layers_num=layers_num, \
                                  lambda_=lambda_, outputshape=y_train.shape[1])
    return loss


result = gp_minimize(objective, search_space, n_calls=n_calls, random_state=42, acq_func='EI')

# 输出最佳参数
print("最佳超參數: {}".format(result.x))
print("最小損失: {}".format(result.fun))


best_params = result.x


neurons_num = best_params[0]
layers_num = best_params[1]
lambda_ = best_params[2 ]

Epoch 1/1000
4/4 [==============================] - 1s 95ms/step - loss: 2.2391 - mse: 0.4529 - val_loss: 1.8880 - val_mse: 0.2403
Epoch 2/1000
4/4 [==============================] - 0s 64ms/step - loss: 1.7668 - mse: 0.1821 - val_loss: 1.6163 - val_mse: 0.1538
Epoch 3/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.5290 - mse: 0.1210 - val_loss: 1.3906 - val_mse: 0.0881
Epoch 4/1000
4/4 [==============================] - 0s 26ms/step - loss: 1.3358 - mse: 0.0800 - val_loss: 1.2312 - val_mse: 0.0652
Epoch 5/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.1776 - mse: 0.0511 - val_loss: 1.1006 - val_mse: 0.0499
Epoch 6/1000
4/4 [==============================] - 0s 27ms/step - loss: 1.0577 - mse: 0.0401 - val_loss: 0.9837 - val_mse: 0.0295
Epoch 7/1000
4/4 [==============================] - 0s 29ms/step - loss: 0.9536 - mse: 0.0272 - val_loss: 0.8972 - val_mse: 0.0238
Epoch 8/1000
4/4 [==============================] - 0s 25ms/step - loss: 0.8692 - m

In [8]:
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import os
import json
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

def convert_to_serializable(obj):
    if isinstance(obj, np.int64):
        return int(obj)
    elif isinstance(obj, l2):
        return {"type": "L2", "l2": obj.l2}
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")
#決定這次的模型編號
model_idx = 1
#本次預測的變數為：slope(隱波與履約價之間的斜率)或iv(隱波的數值)
forecast_variable = 'slope'
#資料設定都相同的情況下，最多儲存多少種模型結果
max_model = 20

batch_size = 500
epochs = 500
model_structure = {
        'input_shape': X_test.shape[1],
        'neurons_num': ([neurons_num] * (layers_num-1)) + [y_test.shape[1]], 
        'kernel_regularizer':[l2(lambda_)] * layers_num,
        'dense_activations': ['relu'] * layers_num
    }

structure_list = [model_structure]
structure_name_list = ['model_structure']
General_structure = {
    structure_name_list[0] : structure_list[0],
}


def save_model_txt(file_path, structure_list, structure_name_list, mse, mse_adj):
    with open('{}'.format(file_path), 'w', encoding='utf-8') as txt_file:
        for i in range(len(structure_list)):
            txt_file.write(structure_name_list[i] + ':\n')
            for key, value in structure_list[i].items():
                txt_file.write(f"{key}: {value}\n")
            txt_file.write('-----------------\n')
        txt_file.write('MSE:{}'.format(mse))
        txt_file.write('MSE_adj:{}'.format(mse_adj))


alltrain = 10
allMSE = np.zeros(alltrain)
allMSE_adj = np.zeros(alltrain)  
allForecastIV = np.zeros((K_num*len(X_test), alltrain))
allSSE_everyday = np.zeros((K_num*len(X_test), alltrain))
allSSE_everyday_adj = np.zeros((K_num*len(X_test), alltrain))
for j in range(alltrain):
    #設定模型結構
    model = set_Linear_series(model_structure)
    model.compile(loss='mse', optimizer = 'adam', metrics=['mse'])  

    # 訓練模型
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=10, \
                               verbose=2, mode='min', restore_best_weights=True)
    hist_model = model.fit(
        X_train, y_train, epochs=epochs, batch_size=batch_size, \
        validation_split=0.2, callbacks=[early_stopping]
    )




    #得到輸出
    y_pred = model.predict(X_test)
    y_pred = y_pred / magnification_slope
    columns_names = ['履約價', '隱含波動率({})'.format(s_c)]
    column_index = [IV_data.columns.get_loc(col) for col in columns_names]
    ForecastIV = np.zeros(((len(y_pred), K_num)))
    SSE_everyday = np.zeros(((len(y_pred), K_num)))
    SSE_everyday_adj = np.zeros(((len(y_pred), K_num)))

    model_name = 'model{}'.format(model_idx)
    #model_file = 'model{}.h5'.format(model_idx)
    model_png = 'model{}.png'.format(model_idx)
    Forecast_name = 'model{}.csv'.format(model_idx)
    model_txt = 'model{}.txt'.format(model_idx)
    model_json = 'model{}.json'.format(model_idx)
    for i in range(len(y_pred)):
        K_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[0]]
        iv_day = IV_matrix_test[i*K_num:(i+1)*K_num, column_index[1]]
        y_pred_day = y_pred[i]
        iv_mean = iv_day / np.mean(iv_day)
        ForecastIV[i], sse_day = minSSE_recovery(y=iv_day, x=K_day, slope_yhat=y_pred_day)
        SSE_everyday[i] = sse_day
        SSE_everyday_adj[i] = sse_day / iv_mean

    ForecastIV = np.reshape(ForecastIV,(-1,1))
    sse_everyday = SSE_everyday[:, 0]
    sse_everyday_adj = SSE_everyday_adj[:, 0]
    SSE_everyday = np.reshape(SSE_everyday ,(-1,1))
    SSE_everyday_adj = np.reshape(SSE_everyday_adj ,(-1,1))
    allMSE[j] = np.mean(sse_everyday)
    allMSE_adj[j] = np.mean(sse_everyday_adj)
    allForecastIV[:,j:j+1] = ForecastIV
    allSSE_everyday[:, j:j+1] = SSE_everyday
    allSSE_everyday_adj[:, j:j+1] = SSE_everyday_adj

min_MSE_idx = np.argsort(allMSE)[0]
ForecastIV = allForecastIV[:, min_MSE_idx:min_MSE_idx+1]
SSE_everyday = allSSE_everyday[:, min_MSE_idx:min_MSE_idx+1]
SSE_everyday_adj = allSSE_everyday_adj[:, min_MSE_idx:min_MSE_idx+1]
min_MSE = allMSE[min_MSE_idx]
min_MSE_adj = allMSE_adj[min_MSE_idx]

Forecast_matrix  = np.hstack((IV_matrix_test, ForecastIV, SSE_everyday, SSE_everyday_adj))
column = np.hstack((IV_data.columns.to_numpy(), \
                        np.array(['上市天數(交易日)','預測隱含波動率({})'.format(s_c), 'loss', '調整後loss'])))
    
   
Forecast_Data = pd.DataFrame(data=Forecast_matrix, columns=column)


loss_columns_names = ['交易日期', '到期天數', '上市天數(交易日)', 'loss']
loss_columns_index = [Forecast_Data.columns.get_loc(col) for col in loss_columns_names]
loss_Data = Forecast_Data.iloc[range(0, len(Forecast_Data), K_num), loss_columns_index]
loss_Data = loss_Data.reset_index().iloc[:,1:]
loss_Data = loss_Data.rename(columns={'loss': model_name})
MSE_data = pd.DataFrame(columns=loss_Data.columns, data=[['MSE', 'MSE', 'MSE', min_MSE]])
loss_Data = pd.concat([loss_Data, MSE_data], axis=0)
loss_Data = loss_Data.reset_index().iloc[:,1:]
print(min_MSE)




model_path = top_path
model_type = 'ANN'

model_Dir_tree = ['Forecast&model', expiry, IV_type, 'K_{}'.format(K_Range_dir), model_type,\
                  'seq{}_seq{}_min{}'.format(seq_length1, seq_length2, min_contract_day), forecast_variable]
for model_dir in model_Dir_tree:
    if model_dir not in os.listdir(model_path):
        os.mkdir(model_path + model_dir)
    model_path = model_path + model_dir + '/'

loss_adj_columns_names = ['交易日期', '到期天數', '調整後loss']
loss_adj_columns_index = [Forecast_Data.columns.get_loc(col) for col in loss_adj_columns_names]
loss_adj_Data = Forecast_Data.iloc[range(0, len(Forecast_Data), K_num), loss_adj_columns_index]
loss_adj_Data = loss_adj_Data.reset_index().iloc[:,1:]
loss_adj_Data = loss_adj_Data.rename(columns={'調整後loss': model_name})
MSE_adj_data = pd.DataFrame(columns=loss_adj_Data.columns, data=[['MSE', 'MSE', min_MSE_adj]])
loss_adj_Data = pd.concat([loss_adj_Data, MSE_adj_data], axis=0)
loss_adj_Data = loss_adj_Data.reset_index().iloc[:,1:]



if 'Modelloss.csv' in os.listdir(model_path):
    Modelloss = pd.read_csv(model_path + 'Modelloss.csv', index_col=False, encoding='Big5')
    Modelloss_adj = pd.read_csv(model_path + 'Modelloss_adj.csv', index_col=False, encoding='Big5')
    if model_name in Modelloss.columns:
        if Modelloss[model_name][len(Modelloss)-1] > min_MSE:
            Modelloss[model_name] = loss_Data[model_name]
            Modelloss_adj[model_name] = loss_adj_Data[model_name]
            Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
            #model.save(model_path + model_file)
            with open(model_path+model_json, 'w', encoding='utf-8') as json_file:
                    json.dump(General_structure, json_file, ensure_ascii=False, indent=4, \
                              default=convert_to_serializable)
            save_model_txt(model_path+model_txt, structure_list, structure_name_list, min_MSE, min_MSE_adj)
            plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=False)
            print('原本的{}已經被替換'.format(model_name))
        else:
            print('此模型未被儲存')
    elif len(Modelloss.columns) >= max_model+2 :
        all_MSE = np.array(Modelloss.iloc[-1, 2:])
        max_MSE = np.max(all_MSE)
        if max_MSE > min_MSE:
            max_MSE_idx = (np.arange(len(all_MSE))[np.equal(all_MSE, max_MSE)])[0] +2
            Modelloss.iloc[:,max_MSE_idx] = loss_Data[model_name]
            Modelloss_adj.iloc[:,max_MSE_idx] = loss_adj_Data[model_name]
            model_name = Modelloss.columns[max_MSE_idx]
            Forecast_name = model_name + '.csv'
            #model_file = model_name + '.h5'
            model_png = model_name + '.png'
            Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
            #model.save(model_path + model_file)
            with open(model_path+model_json, 'w', encoding='utf-8') as json_file:
                    json.dump(General_structure, json_file, ensure_ascii=False, indent=4, \
                              default=convert_to_serializable)
            save_model_txt(model_path+model_txt, structure_list, structure_name_list, min_MSE, min_MSE_adj)
            plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=False)
            print('原本的{}已經被替換'.format(model_name))
        else:
            print('此模型未被儲存')
    else:
        Modelloss = pd.concat([Modelloss, loss_Data[model_name]], axis=1)
        Modelloss = Modelloss.reset_index().iloc[:,1:] 
        Modelloss_adj = pd.concat([Modelloss_adj, loss_Data[model_name]], axis=1)
        Modelloss_adj = Modelloss_adj.reset_index().iloc[:,1:] 
        Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
        #model.save(model_path + model_file)
        with open(model_path+model_json, 'w', encoding='utf-8') as json_file:
                    json.dump(General_structure, json_file, ensure_ascii=False, indent=4, \
                              default=convert_to_serializable)
        save_model_txt(model_path+model_txt, structure_list, structure_name_list, min_MSE, min_MSE_adj)
        plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=False)
        print('此模型已經被儲存為{}'.format(model_name))
else:
    Modelloss = loss_Data
    Modelloss_adj = loss_adj_Data
    Forecast_Data.to_csv(model_path + Forecast_name, index=False, encoding='Big5')
    #model.save(model_path + model_file)
    with open(model_path+model_json, 'w', encoding='utf-8') as json_file:
                    json.dump(General_structure, json_file, ensure_ascii=False, indent=4, \
                              default=convert_to_serializable)
    save_model_txt(model_path+model_txt, structure_list, structure_name_list, min_MSE, min_MSE_adj)
    plot_model(model, to_file=model_path + model_png, show_shapes=True, show_layer_names=False)

    
Modelloss.to_csv(model_path + 'Modelloss.csv', index=False, encoding='Big5')
Modelloss_adj.to_csv(model_path + 'Modelloss_adj.csv', index=False, encoding='Big5')




Epoch 1/500
4/4 [==============================] - 1s 135ms/step - loss: 43.0666 - mse: 43.0636 - val_loss: 65.3078 - val_mse: 65.3048
Epoch 2/500
4/4 [==============================] - 0s 28ms/step - loss: 11.2737 - mse: 11.2707 - val_loss: 23.6136 - val_mse: 23.6106
Epoch 3/500
4/4 [==============================] - 0s 28ms/step - loss: 4.2335 - mse: 4.2305 - val_loss: 10.8072 - val_mse: 10.8042
Epoch 4/500
4/4 [==============================] - 0s 27ms/step - loss: 2.0281 - mse: 2.0251 - val_loss: 5.8178 - val_mse: 5.8148
Epoch 5/500
4/4 [==============================] - 0s 29ms/step - loss: 1.0989 - mse: 1.0958 - val_loss: 3.1045 - val_mse: 3.1015
Epoch 6/500
4/4 [==============================] - 0s 28ms/step - loss: 0.5803 - mse: 0.5773 - val_loss: 1.6376 - val_mse: 1.6346
Epoch 7/500
4/4 [==============================] - 0s 28ms/step - loss: 0.2972 - mse: 0.2941 - val_loss: 0.8704 - val_mse: 0.8673
Epoch 8/500
4/4 [==============================] - 0s 28ms/step - loss: 0.1523 